<a href="https://colab.research.google.com/github/AquilaITOH/ROC_interactive_Colab/blob/main/ROC_interactive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ROC Interactive graph
Google colaboratory version v3.1 (25-03-03)

In [1]:
from ipywidgets import interact
import numpy as np
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import row
output_notebook()

In [4]:
# variable 'sigmaN','sigmaNS'は sigma^2
sigmaN= 1.0
muN = -1
sigmaNS =1.0
muNS = 1
range_min = -7
range_max = 9
step = 0.05
criterion = 0.5

x = np.arange(range_min, range_max, step)

yn = (1./np.sqrt(2*np.pi*sigmaN)) * np.exp(-(x - muN)**2/2/sigmaN) #noise
yns = (1./np.sqrt(2*np.pi*sigmaNS)) * np.exp(-(x - muNS)**2/2/sigmaNS) #noise and signal

# x_del thresould以下のf(x)を0に落とすための等差数例
x_del = np.arange(range_min,criterion,step)

#vf1/vf2/v_fn/v_fsはnp.copyでndarrayをcopyするように変更(2023/2/24)
yfn = np.copy(yn)
for i in range(len(x_del)):
    yfn[i] = 0

yfns = np.copy(yns)
for i in range(len(x_del)):
    yfns[i] = 0

pn = []
for i in range(len(yn)):
    pn.append(sum(yn[i:]*step))

ps = []
for i in range(len(yns)):
    ps.append(sum(yns[i:]*step))

pn_t = []
pn_t.append(sum(yfn*step))

ps_t = []
ps_t.append(sum(yfns*step))

In [6]:
# Probability denstiy function
#In the 3.0.0 bokeh release, the plot_width attribute of a figure was replaced with width (2023/2/24)
p1 = figure(title="Noise and Signal Distributions", height=300, width=600, y_range=(0,0.5), x_range=(-5,7))

# 塗りつぶし
t = p1.patch(x, yfn, color="#2222aa",fill_alpha=0.2, line_width =0 )
u = p1.patch(x, yfns, color="#aa2222", fill_alpha=0.2, line_width=0 )

# plot line
r = p1.line(x, yn, color="#2222aa", line_width=3)
s = p1.line(x, yns, color="#aa2222", line_width=3)
p1.yaxis.axis_label = 'Probability density'

# ROC curve
p2 = figure(title="ROC curve", height=300, width=300)
m = p2.line(pn,ps,color="#a9a9a9")
n = p2.scatter(pn_t,ps_t,size=10,color="#22aa22") #circle -> scatter
p2.xaxis.axis_label = 'FPF'
p2.yaxis.axis_label = 'TPF'

plots = row(p1,p2)

In [7]:
def update(v_sigmaN=1.0, v_muN=-1,v_sigmaNS=1.0, v_muNS=1,v_criterion=0.5):
    global range_min
    global renge_max
    global step

    x = np.arange(range_min, range_max, step)
    #v_n, v_sはノイズ像の確率密度関数と信号+ノイズの確率密度関数
    v_n = (1./np.sqrt(2*np.pi*v_sigmaN)) * np.exp(-(x - v_muN)**2/2/v_sigmaN)
    v_ns = (1./np.sqrt(2*np.pi*v_sigmaNS)) * np.exp(-(x - v_muNS)**2/2/v_sigmaNS)

    r.data_source.data['y'] = v_n
    s.data_source.data['y'] = v_ns

    #v_x_del criterion以下のf(x)を0に落とすためのxの等差数列、最小値range_min, 最大値v_criterion、公比step
    v_x_del = np.arange(range_min,v_criterion,step)

    #noiseのPDFからthreshould以上を塗りつぶしたグラフを作成
    v_fn = np.copy(v_n)
    #criterion以下のf(x)を0に落とす
    for i in range(len(v_x_del)):
        v_fn[i] = 0

    v_fns = np.copy(v_ns)
    for i in range(len(v_x_del)):
        v_fns[i] = 0

    #t noiseのPDFの下、threshold以下を塗りつぶす
    t.data_source.data['y'] = v_fn
    u.data_source.data['y'] = v_fns

    #calculte ROC curve
    #calc cumulative distribution function of noise PDF
    v_pn = []
    for i in range(len(v_n)):
        v_pn.append(sum(v_n[i:]*step))

    v_pns = []
    for i in range(len(v_ns)):
        v_pns.append(sum(v_ns[i:]*step))

    m.data_source.data['x'] = v_pn
    m.data_source.data['y'] = v_pns

    #リストで定義
    v_pn_t = []
    v_pn_t.append(sum(v_fn*step))

    v_pns_t = []
    v_pns_t.append(sum(v_fns*step))


    print("FPF {:.2f}".format(v_pn_t[0])) #v_pn_tはlist
    print("TPF {:.2f}".format(v_pns_t[0]))

#print("TPF"+str(v_pns_t))

    n.data_source.data['x'] = v_pn_t
    n.data_source.data['y'] = v_pns_t

    push_notebook()

In [8]:
show(plots, notebook_handle=True)
interact(update, v_sigmaN=(0.5,5,0.5), v_muN=(-3,4,0.2), v_sigmaNS=(0.5,5,0.5), v_muNS=(-3,4,0.2), v_criterion=(-4,7,0.25))

interactive(children=(FloatSlider(value=1.0, description='v_sigmaN', max=5.0, min=0.5, step=0.5), FloatSlider(…

<function __main__.update(v_sigmaN=1.0, v_muN=-1, v_sigmaNS=1.0, v_muNS=1, v_criterion=0.5)>



 # 参考
 [Bokeh/ Working in the Notebook](http://bokeh.pydata.org/en/latest/docs/user_guide/notebook.html)
  Jupyter Interactors.ipynb example notebook  

 [ipywidgetsとBokeh使ってインタラクティブな可視化をする](http://qiita.com/driller/items/0730325bf5c1cd689979)  
  DataReaderでの株価取得がうまくいかない。google financeでNYSE 3letter tickerならok

  [Titles over layouts a la "suptitle" #5463](https://github.com/bokeh/bokeh/issues/5463) Figureに複数グラフの描画 bokeh.layouts Row  

###  関数のデコレート  
 [Pythonのデコレータについて](http://qiita.com/mtb_beta/items/d257519b018b8cd0cc2e)

##  ipywidgets.interactの概要
 [IPython Notebookをアドホック分析環境として使う](http://zqpythonic.qiniucdn.com/data/20141217180501/index.html)  MatplotlibとBokeh, holdはうまくいかない  
 [Jupyter Notebookを動的に使ってみる](http://qiita.com/y__sama/items/654ed8ab7464718876f9)
  
## ネタ元
  [【統計学】ROC曲線とは何か、アニメーションで理解する。](http://qiita.com/kenmatsu4/items/550b38f4fa31e9af6f4f)

